#Регрессия

##Базовая модель

На первом этапе построения модели регрессии была использована максимально простая постановка задачи. Исходные транзакционные данные были агрегированы до уровня чека: для каждого Invoice вычислялись суммарная стоимость покупки (TotalCheck), общее количество купленных единиц товара (TotalQuantity), число строк в чеке (NumLines) и средняя цена одной единицы (MeanPrice). В качестве базовой модели для регрессии был выбран метод k-ближайших соседей (KNN) с параметрами по умолчанию:
k=5, равномерное взвешивание соседей и евклидово расстояние. Модель обучалась на этом простом наборе признаков без масштабирования, подбора гиперпараметров и дополнительных преобразований. Цель данного шага – получить отправную точку (baseline), с которой далее можно сравнивать улучшенные варианты моделей с расширенным набором признаков и более сложным препроцессингом.

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


DATA_DIR = Path("/content/drive/MyDrive/Учеба/Фреймворки/data")
SIMPLE_REG_PATH = DATA_DIR / "regression_data_simple.csv"

reg_simple = pd.read_csv(SIMPLE_REG_PATH)

print("Simple regression dataset shape:", reg_simple.shape)
print(reg_simple.head())

feature_cols = ["TotalQuantity", "NumLines", "MeanPrice"]
target_col = "TotalCheck"

X = reg_simple[feature_cols].values
y = reg_simple[target_col].values

print("\nFeature matrix shape:", X.shape)
print("Target vector shape:", y.shape)

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
)

print("\nTrain size:", X_train.shape[0])
print("Test size:", X_test.shape[0])

knn_base = KNeighborsRegressor(
    n_neighbors=5,
    weights="uniform",
    metric="minkowski",
    p=2
)

knn_base.fit(X_train, y_train)

y_pred = knn_base.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

r2 = r2_score(y_test, y_pred)

print("\n=== Baseline KNN regression (simple features) ===")
print(f"MAE:  {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²:   {r2:.4f}")


Simple regression dataset shape: (39574, 6)
   Invoice  CustomerID  TotalCheck  TotalQuantity  NumLines  MeanPrice
0   489434     13085.0      505.30            166         8   4.081250
1   489435     13085.0      145.80             60         4   2.625000
2   489436     13078.0      630.33            193        19   3.730526
3   489437     15362.0      310.75            145        23   3.628261
4   489438     18102.0     2286.24            826        17   2.591176

Feature matrix shape: (39574, 3)
Target vector shape: (39574,)

Train size: 31659
Test size: 7915

=== Baseline KNN regression (simple features) ===
MAE:  154.29
RMSE: 521.70
R²:   0.7252


После оценки базовой модели KNN на простом наборе признаков без какого-либо масштабирования следующим шагом было логично проверить влияние нормализации признаков для базовой модели. Для этого была использована та же самая постановка задачи (признаки TotalQuantity, NumLines, MeanPrice и таргет TotalCheck), но перед обучением KNN к признакам применялся StandardScaler, приводящий их к общему масштабу.

In [ ]:
feature_cols = ["TotalQuantity", "NumLines", "MeanPrice"]
target_col = "TotalCheck"

X = reg_simple[feature_cols]
y = reg_simple[target_col].values

print("\nFeature matrix shape:", X.shape)
print("Target vector shape:", y.shape)


X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
)

print("\nTrain size:", X_train.shape[0])
print("Test size:", X_test.shape[0])


knn_scaled = Pipeline(
    steps=[
        ("scaler", StandardScaler()),
        ("model", KNeighborsRegressor(
            n_neighbors=5,
            weights="uniform",
            metric="minkowski",
            p=2,
        )),
    ]
)

knn_scaled.fit(X_train, y_train)


y_pred = knn_scaled.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("\n=== Baseline KNN regression WITH scaling (simple features) ===")
print(f"MAE:  {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²:   {r2:.4f}")



Feature matrix shape: (39574, 3)
Target vector shape: (39574,)

Train size: 31659
Test size: 7915

=== Baseline KNN regression WITH scaling (simple features) ===
MAE:  116.36
RMSE: 479.45
R²:   0.7679


То есть средняя ошибка сократилась, а доля объяснённой вариации целевой переменной выросла. Это ожидаемо для метода k-ближайших соседей, так как он напрямую опирается на расстояния между объектами, и несбалансованные по масштабу признаки могут искажать геометрию пространства. В данном случае можно считать, что масштабирование простых агрегированных признаков даёт более «честное» расстояние между чеками и приводит к улучшению качества базовой модели.

Как мы помним из анализа данных, в распределении целевой переменной TotalCheck присутствуют выраженные выбросы, которые могут сильно искажать картину и ухудшать качество модели. Поэтому дополнительно проверим гипотезу о том, что удаление аномально больших чеков улучшит результаты: для этого отбросим все строки, в которых значение TotalCheck превышает выбранный порог (например, 99.5-й перцентиль распределения).

In [ ]:
q_high = reg_simple["TotalCheck"].quantile(0.995)
mask = reg_simple["TotalCheck"] <= q_high
reg_trim = reg_simple[mask].reset_index(drop=True)

X = reg_trim[feature_cols].values
y = reg_trim[target_col].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

knn_trim = Pipeline(
    steps=[
        ("scaler", StandardScaler()),
        ("model", KNeighborsRegressor(
            n_neighbors=5,
            weights="uniform",
            metric="minkowski",
            p=2,
        )),
    ]
)

knn_trim.fit(X_train, y_train)

y_pred = knn_trim.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("\n=== KNN on trimmed target (TotalCheck <= 99.5 percentile) ===")
print(f"MAE:  {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²:   {r2:.4f}")
print(f"Data size after trim: {len(reg_trim)} rows")



=== KNN on trimmed target (TotalCheck <= 99.5 percentile) ===
MAE:  96.51
RMSE: 227.01
R²:   0.8472
Data size after trim: 39376 rows


Получилось гораздо лучше!

##Улучшенная модель

После базового эксперимента с простым набором признаков (TotalQuantity, NumLines, MeanPrice) и «голым» KNN имеет смысл проверить, насколько качество модели можно улучшить за счёт более содержательного описания чека и более аккуратного препроцессинга.

Для этого используется улучшенный датасет regression_data.csv, в котором для каждого чека добавлены:

агрегированные признаки корзины:

TotalQuantity, NumLines, NumUniqueItems, MeanPrice;

временные признаки:

InvoiceHour, InvoiceDayOfWeek, InvoiceMonth;

флаг IsNewCustomer, отражающий, является ли покупатель новым;

набор признаков формата qty_<…> для top-N наиболее часто встречающихся в чеках товаров, где каждый столбец показывает, сколько единиц данного товара вошло в конкретный чек.

В отличие от базового варианта, при обучении улучшенной модели:

числовые признаки масштабируются с помощью StandardScaler, чтобы привести их к сопоставимому масштабу (для KNN это критично, так как модель основана на расстояниях);

категориальный признак Country кодируется с помощью One-Hot Encoding (OneHotEncoder), чтобы информация о стране также могла учитываться в метрике расстояния;

метаданные (Invoice, CustomerID, InvoiceDate) в обучение не включаются и используются только для идентификации.

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


DATA_DIR = Path("/content/drive/MyDrive/Учеба/Фреймворки/data")
REG_PATH = DATA_DIR / "regression_data_hard.csv"


reg_imp = pd.read_csv(REG_PATH, parse_dates=["InvoiceDate"])

print("Improved regression dataset shape:", reg_imp.shape)
print(reg_imp.head())


exclude_cols = ["TotalCheck", "Invoice", "CustomerID", "InvoiceDate"]
feature_cols = [c for c in reg_imp.columns if c not in exclude_cols]

X = reg_imp[feature_cols]
y = reg_imp["TotalCheck"].values

print("\nFeature columns:", len(feature_cols))
print("Example feature columns:", feature_cols[:10])
print("Feature matrix shape:", X.shape)
print("Target vector shape:", y.shape)


X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
)

print("\nTrain size:", X_train.shape[0])
print("Test size:", X_test.shape[0])


#    - country: OneHotEncoder
numeric_features = [c for c in feature_cols if c != "Country"]
categorical_features = ["Country"]


#    - numeric features: StandardScaler
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_features),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features),
    ]
)

knn_improved = Pipeline(
    steps=[
        ("preprocess", preprocessor),
        ("model", KNeighborsRegressor(
            n_neighbors=5,
            weights="uniform",
            metric="minkowski",
            p=2,
        )),
    ]
)


knn_improved.fit(X_train, y_train)


y_pred = knn_improved.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("\n=== Improved KNN regression (extended features + scaling + OHE Country) ===")
print(f"MAE:  {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²:   {r2:.4f}")


Improved regression dataset shape: (39574, 113)
   Invoice  CustomerID         Country         InvoiceDate  TotalCheck  \
0   491725     12346.0  United Kingdom 2009-12-14 08:34:00        45.0   
1   491742     12346.0  United Kingdom 2009-12-14 11:00:00        22.5   
2   491744     12346.0  United Kingdom 2009-12-14 11:02:00        22.5   
3   492718     12346.0  United Kingdom 2009-12-18 10:47:00        22.5   
4   492722     12346.0  United Kingdom 2009-12-18 10:55:00         1.0   

   TotalQuantity  NumLines  NumUniqueItems  MeanPrice  InvoiceHour  ...  \
0             10         1               1        4.5            8  ...   
1              5         1               1        4.5           11  ...   
2              5         1               1        4.5           11  ...   
3              5         1               1        4.5           10  ...   
4              1         1               1        1.0           10  ...   

   qty_ANTIQUE_SILVER_TEA_GLASS_ETCHED  qty_HANGING_HEAR

Для улучшенного набора признаков была обучена модель KNN с масштабированием числовых признаков и one-hot кодированием страны. На тестовой выборке были получены следующие значения метрик:

MAE ≈ 245.18

RMSE ≈ 1005.49

𝑅
2
≈
0.38
R
2
≈0.38

По сравнению с базовым вариантом (MAE ≈ 154.29, RMSE ≈ 521.70,
𝑅
2
≈
0.73
R
2
≈0.73) качество модели заметно ухудшилось: средняя ошибка возросла, а доля объяснённой вариации целевой переменной существенно снизилась. Это показывает, что для метода KNN добавление большого числа признаков (в том числе разреженных: one-hot по странам и индикаторов для top-N товаров) без отбора и настройки гиперпараметров приводит скорее к «размыванию» метрики расстояния и усилению шума, чем к улучшению качества.

Таким образом, гипотеза о том, что расширение набора признаков автоматически улучшит качество KNN-модели, в данном случае не подтверждается. При этом сформированный улучшенный датасет остаётся полезным: более сложные модели (деревья решений, случайный лес, градиентный бустинг и т.п.) потенциально могут эффективнее использовать эти признаки и демонстрировать выигрыш по качеству по сравнению с простым бейзлайном.

Попробуем убрать параметры "qty_*" и убрать выбросы

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

target_col = "TotalCheck"

# ===== 1. Trim outliers in target =====
# Use 99.5 percentile as upper cutoff for TotalCheck
q_high = reg_imp[target_col].quantile(0.995)
mask = reg_imp[target_col] <= q_high
reg_trim = reg_imp[mask].reset_index(drop=True)

print("Original rows:", len(reg_imp))
print("Trimmed rows:", len(reg_trim))
print("Cutoff (99.5% quantile) TotalCheck:", q_high)

# ===== 2. Define features on trimmed data (no qty_*) =====
qty_cols = [c for c in reg_trim.columns if c.startswith("qty_")]
exclude_cols = ["Invoice", "CustomerID", "InvoiceDate", target_col] + qty_cols

feature_cols = [c for c in reg_trim.columns if c not in exclude_cols]

print("\nFeature columns used for KNN (reduced, trimmed):", len(feature_cols))
print("Example feature columns:", feature_cols[:10])

X = reg_trim[feature_cols]
y = reg_trim[target_col].values

print("X shape:", X.shape)
print("y shape:", y.shape)

# ===== 3. Train / test split =====
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
)

print("\nTrain size:", X_train.shape[0])
print("Test size:", X_test.shape[0])

# ===== 4. Preprocessing: numeric + Country (OHE) =====
numeric_features = [c for c in feature_cols if c != "Country"]
categorical_features = ["Country"] if "Country" in feature_cols else []

transformers = []
if numeric_features:
    transformers.append(("num", StandardScaler(), numeric_features))
if categorical_features:
    transformers.append(("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features))

preprocessor = ColumnTransformer(transformers=transformers)

# ===== 5. KNN model =====
knn_reduced_trimmed = Pipeline(
    steps=[
        ("preprocess", preprocessor),
        ("model", KNeighborsRegressor(
            n_neighbors=5,
            weights="uniform",
            metric="minkowski",
            p=2,
        )),
    ]
)

# ===== 6. Train =====
knn_reduced_trimmed.fit(X_train, y_train)

# ===== 7. Predict & metrics =====
y_pred = knn_reduced_trimmed.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("\n=== KNN regression (reduced features + outlier-trimmed target, no qty_*) ===")
print(f"MAE:  {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²:   {r2:.4f}")


Original rows: 39574
Trimmed rows: 39376
Cutoff (99.5% quantile) TotalCheck: 5739.156000000027

Feature columns used for KNN (reduced, trimmed): 9
Example feature columns: ['Country', 'TotalQuantity', 'NumLines', 'NumUniqueItems', 'MeanPrice', 'InvoiceHour', 'InvoiceDayOfWeek', 'InvoiceMonth', 'IsNewCustomer']
X shape: (39376, 9)
y shape: (39376,)

Train size: 31500
Test size: 7876

=== KNN regression (reduced features + outlier-trimmed target, no qty_*) ===
MAE:  143.39
RMSE: 285.05
R²:   0.7382


По сравнению с первым запуском «продвинутой» модели на расширенных признаках без обрезки выбросов качество заметно улучшилось: средняя ошибка снизилась, разброс предсказаний относительно истинных значений сократился, а доля объяснённой вариации таргета выросла до уровня около 0.74. То есть удаление экстремально больших чеков действительно делает модель более устойчивой и лучше описывающей «типичное» поведение покупателей.

При этом даже после тримминга модель на расширенном наборе признаков остаётся немного хуже по качеству, чем базовая KNN-модель на простых агрегированных признаках без дополнительных фич: для базового варианта со скалированием R² остаётся выше (≈ 0.77), а MAE ниже. Это показывает, что сама по себе сложность признакового пространства не гарантирует выигрыша для KNN.

Далее логично перейти к следующему шагу улучшения: подбору гиперпараметров KNN (число соседей, схема взвешивания, выбор метрики расстояния) на кросс-валидации, чтобы проверить, удастся ли дополнительно повысить качество модели без изменения структуры признаков.

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

target_col = "TotalCheck"

# ===== 1. Trim outliers in target =====
# Use 99.5 percentile as upper cutoff for TotalCheck
q_high = reg_imp[target_col].quantile(0.995)
mask = reg_imp[target_col] <= q_high
reg_trim = reg_imp[mask].reset_index(drop=True)

print("Original rows:", len(reg_imp))
print("Trimmed rows:", len(reg_trim))
print("Cutoff (99.5% quantile) TotalCheck:", q_high)

# ===== 2. Define features on trimmed data (no qty_*) =====
qty_cols = [c for c in reg_trim.columns if c.startswith("qty_")]
exclude_cols = ["Invoice", "CustomerID", "InvoiceDate", target_col] + qty_cols

feature_cols = [c for c in reg_trim.columns if c not in exclude_cols]

print("\nFeature columns used for KNN (reduced, trimmed):", len(feature_cols))
print("Example feature columns:", feature_cols[:10])

X = reg_trim[feature_cols]
y = reg_trim[target_col].values

print("X shape:", X.shape)
print("y shape:", y.shape)

# ===== 3. Train / test split =====
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
)

print("\nTrain size:", X_train.shape[0])
print("Test size:", X_test.shape[0])

# ===== 4. Preprocessing: numeric + Country (OHE) =====
numeric_features = [c for c in feature_cols if c != "Country"]
categorical_features = ["Country"] if "Country" in feature_cols else []

transformers = []
if numeric_features:
    transformers.append(("num", StandardScaler(), numeric_features))
if categorical_features:
    transformers.append(("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features))

preprocessor = ColumnTransformer(transformers=transformers)

# ===== 5. Pipeline with KNN (for GridSearch) =====
base_pipeline = Pipeline(
    steps=[
        ("preprocess", preprocessor),
        ("model", KNeighborsRegressor()),
    ]
)

# ===== 6. Hyperparameter grid =====
param_grid = {
    "model__n_neighbors": [3, 5, 10, 20, 30],
    "model__weights": ["uniform", "distance"],
    "model__p": [1, 2],  # 1 = Manhattan, 2 = Euclidean
}

grid = GridSearchCV(
    estimator=base_pipeline,
    param_grid=param_grid,
    cv=5,
    scoring="neg_mean_absolute_error",
    n_jobs=-1,
)

print("\nRunning GridSearchCV...")
grid.fit(X_train, y_train)

print("Best params:", grid.best_params_)
print("Best CV MAE:", -grid.best_score_)

# ===== 7. Evaluate best model on test set =====
best_model = grid.best_estimator_

y_pred = best_model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("\n=== Tuned KNN regression (reduced features + outlier-trimmed target, no qty_*) ===")
print(f"MAE:  {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²:   {r2:.4f}")


Original rows: 39574
Trimmed rows: 39376
Cutoff (99.5% quantile) TotalCheck: 5739.156000000027

Feature columns used for KNN (reduced, trimmed): 9
Example feature columns: ['Country', 'TotalQuantity', 'NumLines', 'NumUniqueItems', 'MeanPrice', 'InvoiceHour', 'InvoiceDayOfWeek', 'InvoiceMonth', 'IsNewCustomer']
X shape: (39376, 9)
y shape: (39376,)

Train size: 31500
Test size: 7876

Running GridSearchCV...
Best params: {'model__n_neighbors': 10, 'model__p': 1, 'model__weights': 'distance'}
Best CV MAE: 137.2060344450502

=== Tuned KNN regression (reduced features + outlier-trimmed target, no qty_*) ===
MAE:  134.57
RMSE: 271.46
R²:   0.7626


Подбор гиперпараметров (k, метрика, схема взвешивания) на кросс-валидации позволил улучшить качество модели KNN на улучшенном и усечённом датасете: MAE и RMSE снизились, а R² вырос с ~0.74 до ~0.76. Однако даже после тюнинга данная конфигурация не даёт существенного выигрыша по сравнению с гораздо более простой моделью KNN на трёх агрегированных признаках, при том что процедура подбора гиперпараметров является вычислительно затратной (порядка 10 минут на один прогон). Учитывая это, простой KNN с масштабированием признаков остаётся рациональным и эффективным бейзлайном для данной задачи.

##Имплементирование алгоритма

###Базовая модель

Ниже приведена самостоятельная реализация алгоритма KNN на «обычных» данных, сформированных для базового бейзлайна: в качестве признаков используются агрегаты по чеку (TotalQuantity, NumLines, MeanPrice), в качестве целевой переменной ‒ TotalCheck. Модель обучается без масштабирования признаков и без дополнительной обработки выбросов, что позволяет напрямую сравнить результаты кастомной реализации с базовой библиотечной моделью.

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


class MyKNNRegressor:
    def __init__(self, n_neighbors=5, p=2, weights="uniform"):

        self.n_neighbors = n_neighbors
        self.p = p
        self.weights = weights
        self.X_train = None
        self.y_train = None

    def fit(self, X, y):

        X = np.asarray(X)
        y = np.asarray(y)

        if X.ndim != 2:
            raise ValueError("X must be 2D array of shape (n_samples, n_features)")
        if y.ndim != 1:
            raise ValueError("y must be 1D array of shape (n_samples,)")

        if X.shape[0] != y.shape[0]:
            raise ValueError("X and y must have the same number of samples")

        self.X_train = X
        self.y_train = y
        return self

    def _minkowski_distance(self, x):

        diff = self.X_train - x

        if self.p == 1:
            dist = np.sum(np.abs(diff), axis=1)
        elif self.p == 2:
            dist = np.sqrt(np.sum(diff ** 2, axis=1))
        else:
            dist = np.sum(np.abs(diff) ** self.p, axis=1) ** (1.0 / self.p)

        return dist

    def _predict_one(self, x):

        distances = self._minkowski_distance(x)

        k = min(self.n_neighbors, len(distances))
        neighbor_idx = np.argpartition(distances, k - 1)[:k]

        neighbor_y = self.y_train[neighbor_idx]
        neighbor_dist = distances[neighbor_idx]

        if self.weights == "uniform":
            return neighbor_y.mean()
        elif self.weights == "distance":
            eps = 1e-8
            w = 1.0 / (neighbor_dist + eps)
            return np.sum(w * neighbor_y) / np.sum(w)
        else:
            raise ValueError(f"Unknown weights type: {self.weights}")

    def predict(self, X):

        if self.X_train is None or self.y_train is None:
            raise RuntimeError("You must call fit() before predict().")

        X = np.asarray(X)
        if X.ndim == 1:
            X = X.reshape(1, -1)

        preds = []
        for i in range(X.shape[0]):
            y_hat = self._predict_one(X[i])
            preds.append(y_hat)
        return np.array(preds)


feature_cols = ["TotalQuantity", "NumLines", "MeanPrice"]
target_col = "TotalCheck"

X = reg_simple[feature_cols].values
y = reg_simple[target_col].values

print("\nFeature matrix shape:", X.shape)
print("Target vector shape:", y.shape)

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
)

print("\nTrain size:", X_train.shape[0])
print("Test size:", X_test.shape[0])

my_knn = MyKNNRegressor(
    n_neighbors=5,
    p=2,
    weights="uniform",
)

my_knn.fit(X_train, y_train)


y_pred = my_knn.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("\n=== Custom KNN regression (simple features, no scaling) ===")
print(f"MAE:  {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²:   {r2:.4f}")





Feature matrix shape: (39574, 3)
Target vector shape: (39574,)

Train size: 31659
Test size: 7915

=== Custom KNN regression (simple features, no scaling) ===
MAE:  154.29
RMSE: 521.70
R²:   0.7252


Для кастомной реализации KNN без масштабирования были получены значения MAE ≈ 154.29, RMSE ≈ 521.70 и
𝑅
2
≈
0.73
R
2
≈0.73, что практически совпадает с результатами библиотечной реализации KNeighborsRegressor с теми же настройками. Это подтверждает корректность собственной имплементации алгоритма.

###Улучшенная модель

Далее реализаем кастомную модель для улучшенного бэйзлайна

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler


class MyKNNRegressor:
    def __init__(self, n_neighbors=5, p=2, weights="uniform"):

        self.n_neighbors = n_neighbors
        self.p = p
        self.weights = weights
        self.X_train = None
        self.y_train = None

    def fit(self, X, y):

        X = np.asarray(X)
        y = np.asarray(y)

        if X.ndim != 2:
            raise ValueError("X must be 2D array of shape (n_samples, n_features)")
        if y.ndim != 1:
            raise ValueError("y must be 1D array of shape (n_samples,)")

        if X.shape[0] != y.shape[0]:
            raise ValueError("X and y must have the same number of samples")

        self.X_train = X
        self.y_train = y
        return self

    def _minkowski_distance(self, x):

        diff = self.X_train - x

        if self.p == 1:
            dist = np.sum(np.abs(diff), axis=1)
        elif self.p == 2:
            dist = np.sqrt(np.sum(diff ** 2, axis=1))
        else:
            dist = np.sum(np.abs(diff) ** self.p, axis=1) ** (1.0 / self.p)

        return dist

    def _predict_one(self, x):

        distances = self._minkowski_distance(x)

        k = min(self.n_neighbors, len(distances))
        neighbor_idx = np.argpartition(distances, k - 1)[:k]

        neighbor_y = self.y_train[neighbor_idx]
        neighbor_dist = distances[neighbor_idx]

        if self.weights == "uniform":
            return neighbor_y.mean()
        elif self.weights == "distance":
            eps = 1e-8
            w = 1.0 / (neighbor_dist + eps)
            return np.sum(w * neighbor_y) / np.sum(w)
        else:
            raise ValueError(f"Unknown weights type: {self.weights}")

    def predict(self, X):

        if self.X_train is None or self.y_train is None:
            raise RuntimeError("You must call fit() before predict().")

        X = np.asarray(X)
        if X.ndim == 1:
            X = X.reshape(1, -1)

        preds = []
        for i in range(X.shape[0]):
            y_hat = self._predict_one(X[i])
            preds.append(y_hat)
        return np.array(preds)


DATA_DIR = Path("/content/drive/MyDrive/Учеба/Фреймворки/data")
SIMPLE_REG_PATH = DATA_DIR / "regression_data_simple.csv"

reg_simple = pd.read_csv(SIMPLE_REG_PATH)

print("Simple regression dataset shape:", reg_simple.shape)
print(reg_simple.head())

feature_cols = ["TotalQuantity", "NumLines", "MeanPrice"]
target_col = "TotalCheck"

q_high = reg_simple[target_col].quantile(0.995)
mask = reg_simple[target_col] <= q_high
reg_trim = reg_simple[mask].reset_index(drop=True)

print("\nOriginal rows:", len(reg_simple))
print("Trimmed rows:", len(reg_trim))
print("Cutoff (99.5% quantile) TotalCheck:", q_high)

X = reg_trim[feature_cols].values
y = reg_trim[target_col].values

print("\nFeature matrix shape:", X.shape)
print("Target vector shape:", y.shape)

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
)

print("\nTrain size:", X_train.shape[0])
print("Test size:", X_test.shape[0])

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

my_knn = MyKNNRegressor(
    n_neighbors=5,
    p=2,
    weights="uniform",
)

my_knn.fit(X_train_scaled, y_train)

y_pred = my_knn.predict(X_test_scaled)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("\n=== Custom KNN regression (simple features, trimmed target + scaling) ===")
print(f"MAE:  {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²:   {r2:.4f}")


Simple regression dataset shape: (39574, 6)
   Invoice  CustomerID  TotalCheck  TotalQuantity  NumLines  MeanPrice
0   489434     13085.0      505.30            166         8   4.081250
1   489435     13085.0      145.80             60         4   2.625000
2   489436     13078.0      630.33            193        19   3.730526
3   489437     15362.0      310.75            145        23   3.628261
4   489438     18102.0     2286.24            826        17   2.591176

Original rows: 39574
Trimmed rows: 39376
Cutoff (99.5% quantile) TotalCheck: 5739.156000000027

Feature matrix shape: (39376, 3)
Target vector shape: (39376,)

Train size: 31500
Test size: 7876

=== Custom KNN regression (simple features, trimmed target + scaling) ===
MAE:  96.50
RMSE: 227.01
R²:   0.8472


Для кастомной KNN-модели с улучшенным бейзлайном (усечённый по верхнему квантилю таргет TotalCheck + масштабирование признаков) были получены значения MAE ≈ 96.50, RMSE ≈ 227.01 и
𝑅
2
≈
0.85
R
2
≈0.85, что также совпадает с результатами библиотечной реализации при аналогичных настройках. Это дополнительно подтверждает корректность собственной реализации алгоритма и применённого препроцессинга.

#Классификация

##Базовая модель

##Улучшенная модель

##Имплементирование алгоритма

###Базовая модель

###Улучшенная модель